# Tuning Notebook: ensemble_simple
Dummy-Daten, kleine Splits, Stage A (3 Blöcke) + kurze Stage B. Exporte unter `outputs/`.

In [ ]:

import os, sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent   # wenn Notebook in repo/notebooks liegt
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import GlobalConfig, DEFAULT_CORR_SPEC, EWMA_CORR_SPEC, outputs_for_model
from src.tuning import run_stageA, run_stageB
import numpy as np, pandas as pd


In [ ]:
from src.models.ensemble_simple import ForecastModel

In [ ]:

# Outputs sicherstellen
outputs_for_model("ensemble_simple")


In [ ]:

# --- Dummy monthly data ---
rng = np.random.default_rng(7)
T = 160  # klein für Demo
p = 20

dates = pd.date_range("2000-01-01", periods=T, freq="MS")
y = pd.Series(rng.normal(size=T), index=dates, name="delta_ip")
y = 0.5*y.shift(1).fillna(0) + y + rng.normal(scale=0.3, size=T)
X = pd.DataFrame(rng.normal(size=(T,p)), index=dates, columns=[f"f{i:03d}" for i in range(p)])
for j in range(min(5, p)):
    X.iloc[:, j] = y.shift(1).fillna(0).values + rng.normal(scale=0.5, size=T)

y.head(), X.head()


In [ ]:

# --- Config ---
cfg = GlobalConfig()
cfg.corr_spec = dict(EWMA_CORR_SPEC)  # global, einheitlich
cfg.lag_candidates = tuple(range(1, 4))
cfg.top_k_lags_per_feature = 1
cfg.k1_topk = 20
cfg.redundancy_method = "greedy"
cfg.redundancy_param = 0.9
cfg.dr_method = "none"   # Modell-spezifisch ggf. im Grid ändern
cfg.nuisance_seasonal = "off"

# Kleine Blöcke
cfg.W0_A = 60
cfg.BLOCKS_A = [(61, 70), (71, 80), (81, 90)]
cfg.W0_B = 100
cfg.policy_window = 6
cfg.policy_gain_min = 0.05
cfg.policy_cooldown = 2


In [ ]:

# Einfaches Ensemble aus lgbm + xgb
model_grid = [
    {"members":[
        {"type":"lgbm","hp":{"learning_rate":0.05,"n_estimators":150,"num_leaves":15}},
        {"type":"xgb","hp":{"learning_rate":0.05,"n_estimators":150,"max_depth":3}}
    ], "weights": None},
    {"members":[
        {"type":"lgbm","hp":{"learning_rate":0.10,"n_estimators":200,"num_leaves":31}},
        {"type":"xgb","hp":{"learning_rate":0.03,"n_estimators":250,"max_depth":2}}
    ], "weights": None},
]


In [ ]:

# --- Stage A ---
shortlist = run_stageA(
    model_name="ensemble_simple",
    model_ctor=lambda hp: ForecastModel(hp),
    model_grid=model_grid,
    X=X, y=y, cfg=cfg
)
shortlist


In [ ]:

# --- Stage B (kurz) ---
run_stageB(
    model_name="ensemble_simple",
    model_ctor=lambda hp: ForecastModel(hp),
    shortlist=shortlist,
    X=X, y=y, cfg=cfg,
    max_months=6
)
print("Stage B done → outputs/stageB/ensemble_simple/")
